In [1]:
import torch
import torch.nn as nn

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import numpy as np
from tqdm.auto import tqdm

In [4]:
path_turing = './unilmv2/'

In [5]:
import sys
sys.path.insert(0, './PLM-NR')

In [6]:
from tnlrv3.modeling import TuringNLRv3ForSequenceClassification
from tnlrv3.configuration_tnlrv3 import TuringNLRv3Config
from tnlrv3.tokenization_tnlrv3 import TuringNLRv3Tokenizer

In [7]:
tokenizer = TuringNLRv3Tokenizer.from_pretrained(os.path.join(path_turing, 'unilm2-base-uncased-vocab.txt'),
                                            do_lower_case=True)

Calling TuringNLRv3Tokenizer.from_pretrained() with the path to a single file or url is deprecated


In [8]:
MODEL_CLASSES = {
    'tnlrv3': (TuringNLRv3Config, TuringNLRv3ForSequenceClassification, TuringNLRv3Tokenizer),
}

# Pretrain Dataset

In [9]:
MAX_TITLE_LEN = 24
MAX_BODY_LEN = 512
COEF = 1

In [10]:
file = './docs_filter.tsv'
with open(file, encoding='utf-8') as f:
    total_lines = f.readlines()

In [11]:
len(total_lines)

1975767

In [12]:
titles, bodies = [], []
for line in total_lines:
    splited = line.strip('\n').split('\t')
    titles.append(splited[3])
    bodies.append(splited[4])

# Pretrain Dataset

In [13]:
from torch.utils.data import DataLoader, Dataset
import random

In [14]:
NPRATIO=9
BATCH_SIZE=32

In [ ]:
import pickle
# Must run Domain-specific_Post-train.ipynb first
with open('./teacher_title_emb.pkl', 'rb') as f:
    title_scoring = pickle.load(f)
with open('./teacher_body_emb.pkl', 'rb') as f:
    body_scoring = pickle.load(f)

In [ ]:
class DistillDataset(Dataset):
    def __init__(self, titles, bodies, teacher_titles, teacher_bodies):
        self.titles = titles
        self.bodies = bodies
        self.teacher_titles = teacher_titles
        self.teacher_bodies = teacher_bodies
        self.len = len(titles)
        
    def __getitem__(self, idx):
        select_list = list(range(0, idx)) + list(range(idx+1, self.len))
        neg_idx = random.sample(select_list, NPRATIO)
        neg_titles = [self.titles[i] for i in neg_idx]
        pos_title = self.titles[idx]
        titles = [tokenizer(title, max_length=MAX_TITLE_LEN, pad_to_max_length=True,
                            truncation=True) for title in [pos_title] + neg_titles]
        input_titles = np.array([title['input_ids'] + title['attention_mask'] for title in titles])
        body = tokenizer(self.bodies[idx], max_length=MAX_BODY_LEN, pad_to_max_length=True,
                        truncation=True)
        input_body = np.array(body['input_ids'] + body['attention_mask'])
                                 
        total_idx = [idx] + neg_idx
        input_teacher_title = self.teacher_titles[total_idx]
        input_teacher_body = self.teacher_bodies[idx]
        label=0
        return input_titles, input_body, label, input_teacher_title, input_teacher_body

    def __len__(self):
        return self.len

In [ ]:
distill_ds = DistillDataset(titles, bodies, title_scoring, body_scoring)

In [ ]:
distill_dl = DataLoader(distill_ds, batch_size=BATCH_SIZE, num_workers=32, shuffle=True, pin_memory=True)

# Pretrain Model

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, emb_size, hidden_size):
        super(AttentionPooling, self).__init__()
        self.att_fc1 = nn.Linear(emb_size, hidden_size)
        self.att_fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x, attn_mask=None):
        """
        Args:
            x: batch_size, candidate_size, emb_dim
            attn_mask: batch_size, candidate_size
        Returns:
            (shape) batch_size, emb_dim
        """
        bz = x.shape[0]
        e = self.att_fc1(x)
        e = nn.Tanh()(e)
        alpha = self.att_fc2(e)
        alpha = torch.exp(alpha)

        if attn_mask is not None:
            alpha = alpha * attn_mask.unsqueeze(2)
        
        alpha = alpha / (torch.sum(alpha, dim=1, keepdim=True) + 1e-8)
        x = torch.bmm(x.permute(0, 2, 1), alpha).squeeze(dim=-1)
        return x

class NewsEncoder(nn.Module):
    def __init__(self, args):
        super(NewsEncoder, self).__init__()
        config_class, model_class, tokenizer_class = MODEL_CLASSES['tnlrv3']
        self.bert_config = config_class.from_pretrained(
            os.path.join(path_turing, 'unilm2-base-uncased-config.json'), 
            output_hidden_states=True,
            num_hidden_layers=args.num_hidden_layers)
        self.bert_model = model_class.from_pretrained(
            os.path.join(path_turing, 'unilm2-base-uncased.bin'), config=self.bert_config)
        self.attn = AttentionPooling(self.bert_config.hidden_size, args.news_query_vector_dim)
        self.dense = nn.Linear(self.bert_config.hidden_size, args.news_dim)

    def forward(self, x):
        '''
            x: batch_size, word_num * 2
            mask: batch_size, word_num
        '''
        batch_size, num_words = x.shape
        num_words = num_words // 2
        text_ids = torch.narrow(x, 1, 0, num_words)
        text_attmask = torch.narrow(x, 1, num_words, num_words)
        word_vecs = self.bert_model(text_ids, text_attmask)[3][self.bert_config.num_hidden_layers]
        news_vec = self.attn(word_vecs)
        news_vec = self.dense(news_vec)
        return news_vec

In [ ]:
class TitleBodySimModel(nn.Module):
    def __init__(self, args):
        super(TitleBodySimModel, self).__init__()
        self.news_encoder = NewsEncoder(args)
        
    def forward(self, title, body):
        '''
            title: bz, 1+K, MAX_TITLE_WORD * 2
            body: bz, MAX_BODY_WORD * 2
            labels: bz
        '''
        body_emb = self.news_encoder(body)             #bz,emb_dim
        bz, candi_num, input_num = title.shape
        title = title.reshape(-1, input_num)
        title_emb = self.news_encoder(title)
        title_emb = title_emb.reshape(bz, candi_num, -1) #bz, 1+K, emb_dim
        
        scores = torch.bmm(title_emb, body_emb.unsqueeze(dim=-1)).squeeze(-1)
        
        return scores, title_emb, body_emb   

In [ ]:
def kd_ce_loss(logits_S, logits_T, temperature=1):
    '''
    Calculate the cross entropy between logits_S and logits_T
    :param logits_S: Tensor of shape (batch_size, length, num_labels) or (batch_size, num_labels)
    :param logits_T: Tensor of shape (batch_size, length, num_labels) or (batch_size, num_labels)
    :param temperature: A float or a tensor of shape (batch_size, length) or (batch_size,)
    '''
    beta_logits_T = logits_T / temperature
    beta_logits_S = logits_S / temperature
    p_T = F.softmax(beta_logits_T, dim=-1)
    loss = -(p_T * F.log_softmax(beta_logits_S, dim=-1)).sum(dim=-1).mean()
    return loss

def hid_mse_loss(state_S, state_T, mask=None):
    '''
    * Calculates the mse loss between `state_S` and `state_T`, which are the hidden state of the models.
    * If the `inputs_mask` is given, masks the positions where ``input_mask==0``.
    * If the hidden sizes of student and teacher are different, 'proj' option is required in `inetermediate_matches` to match the dimensions.
    :param torch.Tensor state_S: tensor of shape  (*batch_size*, *length*, *hidden_size*)
    :param torch.Tensor state_T: tensor of shape  (*batch_size*, *length*, *hidden_size*)
    :param torch.Tensor mask:    tensor of shape  (*batch_size*, *length*)
    '''
    if mask is None:
        loss = F.mse_loss(state_S, state_T)
    else:
        valid_count = mask.sum() * state_S.size(-1)
        loss = (F.mse_loss(state_S, state_T, reduction='none') * mask.unsqueeze(-1)).sum() / valid_count
    return loss

In [ ]:
class DistillModel(nn.Module):
    def __init__(self, args):
        super(DistillModel, self).__init__()
        self.student = TitleBodySimModel(args)
        self.target_loss = nn.CrossEntropyLoss()
        self.transform = nn.Linear(args.news_dim, args.news_dim)
        self.distill_loss = kd_ce_loss
        self.emb_loss = hid_mse_loss
        
        nn.init.xavier_uniform_(self.transform.weight, gain=1.)
        nn.init.constant_(self.transform.bias, 0.0)
    
    def forward(self, title, body, labels, teacher_title, teacher_body):
        student_score, student_title, student_body = self.student(title, body)
        teacher_score = torch.bmm(teacher_title, teacher_body.unsqueeze(dim=-1)).squeeze(dim=-1)
        
        target_loss = self.target_loss(student_score, labels)
        distill_loss = self.distill_loss(student_score, teacher_score)
        emb_loss = self.emb_loss(student_title, self.transform(teacher_title)) + \
                    self.emb_loss(student_body, self.transform(teacher_body))
        
        loss = COEF * target_loss + distill_loss + emb_loss
        return loss, target_loss, distill_loss, emb_loss, student_score

In [ ]:
class Args:
    def __init__(self):
        self.news_query_vector_dim = 200
        self.drop_rate = 0.2
        self.news_dim = 256
        self.num_hidden_layers = 4

In [ ]:
args = Args()

In [ ]:
distill_model = DistillModel(args)

In [ ]:
device = torch.device('cuda')

In [ ]:
distill_model.to(device)

In [ ]:
def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
for param in distill_model.student.news_encoder.bert_model.parameters():
    param.requires_grad = False

for index, layer in enumerate(distill_model.student.news_encoder.bert_model.bert.encoder.layer):
    if index in [2, 3]:
        for param in layer.parameters():
            param.requires_grad = True

In [ ]:
for name, p in distill_model.named_parameters():
    print(name, p.requires_grad)

In [ ]:
rest_param = filter(
    lambda x: id(x) not in list(map(id, distill_model.student.news_encoder.bert_model.parameters())), distill_model.parameters())

optimizer = optim.Adam(
    [{'params':distill_model.student.news_encoder.bert_model.parameters(),'lr':1e-6},
    {'params':rest_param,'lr':1e-5}]
)

In [ ]:
from tqdm import tqdm

In [ ]:
for ep in range(1):
    loss, target_loss, distill_loss, emb_loss = 0.0, 0.0, 0.0, 0.0
    accuary = 0.0
    cnt = 1
    tqdm_util = tqdm(distill_dl)
    distill_model.train()
    for title,body,labels,teacher_title,teacher_body in tqdm_util:
        title = title.cuda(non_blocking=True)
        body = body.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        teacher_title = teacher_title.cuda(non_blocking=True)
        teacher_body = teacher_body.cuda(non_blocking=True)
        
        bz_loss, t_loss, d_loss, e_loss, y_hat = distill_model(title, body, labels, teacher_title, teacher_body)
        
        loss += bz_loss.data.float()
        target_loss += t_loss.data.float()
        distill_loss += d_loss.data.float()
        emb_loss += e_loss.data.float()
        accuary += acc(labels, y_hat)
        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()
        
        if cnt % 10 == 0:
            tqdm_util.set_description('ed: {}, loss: {:.5f}, t_loss: {:.5f}, d_loss: {:.5f}, e_loss: {:.5f}, acc: {:.5f}'.format(
                cnt * BATCH_SIZE, loss.data / cnt, target_loss.data / cnt, distill_loss.data / cnt, emb_loss.data / cnt, accuary / cnt))
        
        cnt += 1
        
    ckpt_path = f'./first_stage_4_layer.pt'
    torch.save({'model_state_dict': distill_model.state_dict()}, ckpt_path)
